Primero importamos las librerias necesarias

In [1]:
import tweepy,json
import pandas as pd
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Stream
from tweepy.streaming import StreamListener
import time
import sys

Abrimos el archivo JSON que contiene los tweets, en el nombre del archivo se indica la fecha y la hora en la que es recogido

In [2]:
tweets = []
for line in open('streamer_20190216-233316.json', 'r'):
    if line.strip():
        tweets.append(json.loads(line))
for line in open('streamer_20190217-205512.json', 'r'):
    if line.strip():
        tweets.append(json.loads(line))


In [4]:
len(tweets)

26895

In [4]:
lista_entendidos = []
for tweet in tweets:
    if 'extended_tweet' in tweet:
        lista_entendidos.append(tweet)
len(lista_entendidos)/len(tweets)*100

14.389291689905187

In [11]:
lista_influencers = []
tweets[0]

{'created_at': 'Sat Feb 16 22:33:12 +0000 2019',
 'id': 1096900327931068416,
 'id_str': '1096900327931068416',
 'text': '@sanchezcastejon No me puedo creer que tu indigencia intelectual no te de ni para entender que has sido un accident… https://t.co/w4Ld7dyvN4',
 'display_text_range': [17, 140],
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'truncated': True,
 'in_reply_to_status_id': 1096711767982567424,
 'in_reply_to_status_id_str': '1096711767982567424',
 'in_reply_to_user_id': 68740712,
 'in_reply_to_user_id_str': '68740712',
 'in_reply_to_screen_name': 'sanchezcastejon',
 'user': {'id': 899100235028459521,
  'id_str': '899100235028459521',
  'name': 'C.Drexler',
  'screen_name': 'ClydeDrexlerB',
  'location': None,
  'url': None,
  'description': None,
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 149,
  'friends_count': 546,
  'listed_count': 1,
  'favourites_count': 5221,
  'statu

In [5]:
diccionario_tweets = {}
for x in tweets: 
    for k,v in tweets[x].items():
        diccionario_tweets[k] = v
diccionario_tweets


TypeError: list indices must be integers or slices, not dict

In [77]:
def flatten_tweets(tweets_json):
    """ Flattens out tweet dictionaries so relevant JSON
        is in a top-level dictionary."""
    tweets_list = []
    
    # Iterate through each tweet
    for tweet in tweets_json:
        tweet_obj = json.loads(tweet)
    
        # Store the user screen name in 'user-screen_name'
        tweet_obj['user-screen_name'] = tweet_obj['user']['screen_name']
    
        # Check if this is a 140+ character tweet
        if 'extended_tweet' in tweet_obj:
            # Store the extended tweet text in 'extended_tweet-full_text'
            tweet_obj['extended_tweet-full_text'] = tweet_obj['extended_tweet']['full_text']
    
        if 'retweeted_status' in tweet_obj:
            # Store the retweet user screen name in 'retweeted_status-user-screen_name'
            tweet_obj['retweeted_status-user-screen_name'] = tweet_obj['retweeted_status']['user']['screen_name']

            # Store the retweet text in 'retweeted_status-text'
            tweet_obj['retweeted_status-text'] = tweet_obj['retweeted_status']['text']
            
        tweets_list.append(tweet_obj)
    return tweets_list

In [6]:
df = pd.DataFrame(tweets)
df.columns

Index(['contributors', 'coordinates', 'created_at', 'display_text_range',
       'entities', 'extended_entities', 'extended_tweet', 'favorite_count',
       'favorited', 'filter_level', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'possibly_sensitive', 'quote_count', 'quoted_status',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink',
       'reply_count', 'retweet_count', 'retweeted', 'retweeted_status',
       'source', 'text', 'timestamp_ms', 'truncated', 'user'],
      dtype='object')